# Build an Adaptive ReAct AI Agent with Flyte

Welcome to this hands-on tutorial on building **adaptive** AI agents using the ReAct (Reason + Act) pattern!

## 🎯 What We'll Build

An **adaptive agent system** that:
- **Thinks** before each action (reasons about what to do)
- **Acts** by calling specialized agents
- **Observes** the results
- **Reflects** on what it learned
- **Adapts** its strategy based on results

Unlike static planners that create a full plan upfront, ReAct agents **adjust their approach** based on what they discover!

## 📚 Learning Objectives

By the end of this tutorial, you'll understand:

1. **ReAct Pattern** - How Reason + Act differs from static planning
2. **Adaptive Execution** - Changing strategy based on observations
3. **Iterative Reasoning** - Building context across multiple steps
4. **Reflection** - Learning from each action to improve next steps
5. **When to Use ReAct** - Comparing static planning vs. adaptive agents

---

## 🏗️ ReAct vs. Static Planning

### Static Planner (from previous tutorial):
```
User Request → Planner creates FULL plan → Execute all steps
```
✅ **Pros:** Parallel execution, efficient for known tasks  
❌ **Cons:** Can't adapt if results are unexpected

### ReAct Agent (this tutorial):
```
User Goal → Think → Act (ONE step) → Observe → Reflect → Think → ...
```
✅ **Pros:** Adapts to results, handles uncertainty, explores solutions  
❌ **Cons:** Sequential execution (slower for independent tasks)

---

## 🔄 The ReAct Loop

```
┌─────────────────────────────────────┐
│         GOAL: Find X and Y          │
└──────────────┬──────────────────────┘
               ↓
     ┌─────────────────────┐
     │  1. THOUGHT          │  "I should search for X first"
     └──────────┬───────────┘
                ↓
     ┌─────────────────────┐
     │  2. ACTION           │  Call web_search_agent("X")
     └──────────┬───────────┘
                ↓
     ┌─────────────────────┐
     │  3. OBSERVATION      │  "Found: X = 42"
     └──────────┬───────────┘
                ↓
     ┌─────────────────────┐
     │  4. REFLECTION       │  "Good! Now I need Y"
     └──────────┬───────────┘
                ↓
     ┌─────────────────────┐
     │  1. THOUGHT          │  "Now search for Y"
     └──────────┬───────────┘
                ↓
              [Loop continues...]
                ↓
     ┌─────────────────────┐
     │  GOAL ACHIEVED!      │  "X=42, Y=17"
     └─────────────────────┘
```

**Key Insight:** Each action informs the next thought. The agent can change its mind!

## ⚙️ Setup and Prerequisites

This tutorial builds on the same infrastructure as the dynamic workflow:
- Same agents (math, string, web_search, code, weather)
- Same tools (from `tools/` directory)
- Same Flyte configuration

**What's NEW:** The orchestration pattern - how we coordinate agents!

In [1]:
# Utility to display code files with syntax highlighting
def print_code_file(file_path):
    from pygments import highlight
    from pygments.lexers import PythonLexer
    from pygments.formatters import HtmlFormatter
    from IPython.display import HTML

    code = open(file_path).read()
    formatter = HtmlFormatter(style="one-dark", noclasses=True)
    html = highlight(code, PythonLexer(), formatter)
    display(HTML(html))

---

## 🧠 The ReAct Workflow - Core Concepts

Let's examine the key components of the ReAct pattern:

### 1️⃣ **Data Models**

We track each step of the reasoning loop:

```python
@dataclass
class ReActStep:
    step_number: int
    thought: str          # "I should search for X"
    action_agent: str     # "web_search"
    action_task: str      # "Find information about X"
    observation: str      # "X is 42"
    reflection: str       # "Great! Now I need Y"

@dataclass
class ReActResult:
    goal: str
    steps: List[ReActStep]  # Full execution trace
    final_answer: str
    total_steps: int
    goal_achieved: bool
```

**Why this matters:** We maintain a **complete audit trail** of the agent's reasoning process!

### 2️⃣ **The Reasoning Prompt**

This is where the magic happens. We ask the LLM to think like a ReAct agent:

```python
system_msg = f"""
You are a ReAct agent using the Reason + Act pattern.

Goal: {user_goal}

Previous steps:
{history_text}

Your task: Decide what to do next to achieve the goal.

Respond in JSON format:
{{
  "thought": "Your reasoning about what to do next and why",
  "action_agent": "agent_name",
  "action_task": "specific task for the agent",
  "goal_achieved": false,
  "final_answer": null
}}

OR if the goal is achieved:
{{
  "thought": "Why I believe the goal is now achieved",
  "goal_achieved": true,
  "final_answer": "The complete answer"
}}
"""
```

**Key Design Choices:**
- ✅ Include previous steps for context (last 3 steps)
- ✅ Force JSON output for structured decisions
- ✅ Require explicit reasoning ("thought" field)
- ✅ One action at a time (no parallel planning)

### 3️⃣ **The Iteration Loop**

Each iteration follows this pattern:

```python
for step_num in range(1, max_steps + 1):
    # 1. Build context from previous steps
    history_text = format_previous_steps(context_history[-3:])
    
    # 2. Ask LLM: "What should we do next?"
    decision = await llm.decide_next_action(
        goal=user_goal,
        history=history_text
    )
    
    # 3. Check if done
    if decision["goal_achieved"]:
        break
    
    # 4. Execute the chosen action
    observation = await execute_agent(
        agent=decision["action_agent"],
        task=decision["action_task"]
    )
    
    # 5. Reflect on the result
    reflection = await llm.reflect(
        action=decision,
        observation=observation
    )
    
    # 6. Record this step
    steps.append(ReActStep(...))
    context_history.append({...})
```

**Notice:** We only execute ONE action per iteration, then reassess!

### 4️⃣ **Reflection - Learning from Results**

After each action, we ask the agent to reflect:

```python
reflection_prompt = f"""
Based on this action and result, reflect on:
1. Was this action helpful?
2. Did we get the information we need?
3. Are we closer to the goal?

Action: {action_agent} - {action_task}
Result: {observation}

Provide a brief reflection (1-2 sentences).
"""
```

**Why reflect?**
- Builds understanding across steps
- Helps LLM learn from mistakes
- Creates richer context for next thought
- Improves decision quality over time

---

## 📝 The Complete ReAct Workflow

Let's look at the full implementation:

### Key Features:

1. **Adaptive Planning** - Each step can change the strategy
2. **Context Accumulation** - Previous steps inform future decisions
3. **Robust JSON Parsing** - Handles LLM output variations
4. **Early Termination** - Stops when goal is achieved
5. **Full Traceability** - Every thought, action, and reflection is logged

### Code Walkthrough:

In [2]:
print_code_file("workflows/flyte_react.py")

---

## 🔍 Code Deep Dive: Critical Sections

Let's examine the most important parts:

### 🧩 Building Context from History

```python
if context_history:
    history_text = "\n\n".join([
        f"Step {s['step']}: {s['thought']}\n"
        f"Action: {s['action_agent']} - {s['action_task']}\n"
        f"Result: {s['observation']}\n"
        f"Reflection: {s['reflection']}"
        for s in context_history[-3:]  # Last 3 steps
    ])
```

**Why last 3 steps?**
- Balances context richness vs. token cost
- Recent steps are most relevant
- Prevents context window overflow

---

### 🎯 Routing to Agents

```python
if action_agent == "math":
    result = await math_agent(action_task)
    observation = result.final_result
elif action_agent == "web_search":
    result = await web_search_agent(action_task)
    observation = getattr(result, 'summary', result.final_result)
```

**Notice:** We use the `summary` for web search to keep context manageable!

---

### 🛡️ Robust JSON Parsing

```python
try:
    decision = json.loads(raw_response)
except json.JSONDecodeError:
    # Extract from markdown code blocks
    json_match = re.search(r'```(?:json)?\s*(\{.*?\})\s*```', raw_response, re.DOTALL)
    if json_match:
        decision = json.loads(json_match.group(1))
    else:
        # Find any JSON object in response
        json_match = re.search(r'\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}', raw_response, re.DOTALL)
```

**Why so defensive?**
- LLMs sometimes add markdown formatting
- Different models have different output styles
- Better to handle gracefully than fail

---

## 🚀 Running the ReAct Workflow

### Local Execution (Development):
```bash
python -m workflows.flyte_react --local --request "Your goal here" --max-steps 10
```

### Remote Execution (Production):
```bash
python -m workflows.flyte_react --request "Your goal here" --max-steps 10
```

### 📝 Example Prompts to Try:

#### Simple Sequential Task:
```python
"Calculate 5 factorial, then count the letters in the result"
```
**Expected behavior:**
- Step 1: Use math agent → "120"
- Step 2: Use string agent → "3 letters"

---

#### Research Task (Adaptive!):
```python
"Find the GDP of France and Germany, then compare them"
```
**Expected behavior:**
- Step 1: Search for France GDP
- Step 2: Search for Germany GDP
- Step 3: Use code agent to compare

**Notice:** The agent discovers it needs 3 steps, not planned upfront!

---

#### Complex Multi-Agent:
```python
"Search for the latest Python version, then write code to check if it's greater than 3.10"
```
**Expected behavior:**
- Step 1: Web search for Python version
- Step 2: Extract version number from search results
- Step 3: Write code to compare versions

**Adaptive behavior:** If search results are unclear, agent might try a different search!

Let's run an example:

In [3]:
# Run a simple ReAct workflow
!python -m workflows.flyte_react --local --request "Calculate 5 factorial, then count the letters in the result" --max-steps 5

Running workflow LOCALLY with flyte.init()

=== ReAct Multi-Agent Workflow ===
Goal: Calculate 5 factorial, then count the letters in the result
Max steps: 5

ReAct WORKFLOW - Goal: Calculate 5 factorial, then count the letters in the result

STEP 1

[ReAct] Reasoning about next action...

💭 Thought: To achieve the goal, I first need to calculate 5 factorial. This is a mathematical task.

🎯 Action: Call math agent
📋 Task: Calculate 5 factorial
[Math Agent] Processing: Calculate 5 factorial
16:23:37.117724 WARNING  models.py:366 - [6c7ef6e3-b580-4dfa-bb5d-772d17bbd511] 
                         Function multiply has parameter a without type         
                         annotation. Data will be pickled.                      
16:23:37.121888 WARNING  models.py:366 - [6c7ef6e3-b580-4dfa-bb5d-772d17bbd511] 
                         Function multiply has parameter b without type         
                         annotation. Data will be pickled.                      
16:23:37.124011 WAR

In [4]:
# Run a simple ReAct workflow
!python -m workflows.flyte_react --request "Calculate 5 factorial, then count the letters in the result" --max-steps 5

Running workflow REMOTELY with flyte.init_from_config()

=== ReAct Multi-Agent Workflow ===
Goal: Calculate 5 factorial, then count the letters in the result
Max steps: 5

16:24:05.880305 WARNING  remote_builder.py:89 -  Image                          
                         356633062068.dkr.ecr.us-east-2.amazonaws.com/union/demo
                         :flyte-243e55e378ea3dad56a6d61b89c220f0 found. Skip    
                         building.                                              
16:24:05.884496 WARNING  _deploy.py:257 -  Built Image for environment base_env,
                         image:                                                 
                         356633062068.dkr.ecr.us-east-2.amazonaws.com/union/demo
                         :flyte-243e55e378ea3dad56a6d61b89c220f0                

Execution: rqg26mjpl5l6tmlhj7dz
URL: https://demo.hosted.unionai.cloud/v2/runs/project/flytesnacks/domain/development/rqg26mjpl5l6tmlhj7dz
Click the link above to view execution d

---

## 📊 ReAct in Action: Step-by-Step Example

Let's trace through a real execution:

**Goal:** "Find France GDP and Germany GDP, then compare them"

### Step 1:
```json
{
  "thought": "I need to find the GDP of France first",
  "action_agent": "web_search",
  "action_task": "France GDP",
  "goal_achieved": false
}
```
**Observation:** "France GDP is $2.78 trillion (2023)"

**Reflection:** "Good! I found France's GDP. Now I need Germany's."

---

### Step 2:
```json
{
  "thought": "Now I need Germany's GDP to compare",
  "action_agent": "web_search",
  "action_task": "Germany GDP",
  "goal_achieved": false
}
```
**Observation:** "Germany GDP is $4.08 trillion (2023)"

**Reflection:** "Perfect! I have both values. Now I can compare them."

---

### Step 3:
```json
{
  "thought": "I have both GDP values. I should compare them.",
  "action_agent": "code",
  "action_task": "Compare France GDP ($2.78T) vs Germany GDP ($4.08T)",
  "goal_achieved": false
}
```
**Observation:** "Germany's GDP ($4.08T) is 46.8% larger than France's ($2.78T)"

**Reflection:** "I now have a complete comparison. Goal achieved!"

---

### Step 4:
```json
{
  "thought": "I have all the information needed to answer the question",
  "goal_achieved": true,
  "final_answer": "France GDP: $2.78 trillion, Germany GDP: $4.08 trillion. Germany's economy is 46.8% larger."
}
```

**Notice how the agent:**
1. ✅ Broke down the problem step-by-step
2. ✅ Used different agents for different tasks
3. ✅ Built context across steps
4. ✅ Decided when the goal was achieved

---

## 🎓 Key Takeaways

### What Makes ReAct Special:

1. **🔄 Adaptive Behavior**
   - Can change strategy based on results
   - No upfront plan required
   - Handles uncertainty gracefully

2. **🧠 Explicit Reasoning**
   - Every action has a "thought" explaining why
   - Reflection builds understanding
   - Full audit trail of decision-making

3. **📚 Context Accumulation**
   - Previous steps inform future decisions
   - Learning happens during execution
   - Can recover from mistakes

4. **🎯 Goal-Oriented**
   - Continuously checks if goal is achieved
   - Can stop early if done
   - More efficient than fixed step counts

---

### When to Use ReAct vs. Static Planning:

| Scenario | Best Approach | Why |
|----------|---------------|-----|
| Known workflow | Static Planner | Parallel execution, faster |
| Uncertain inputs | **ReAct** | Can adapt to unexpected results |
| Research tasks | **ReAct** | Iterative discovery process |
| Independent tasks | Static Planner | Leverage parallelism |
| Exploratory goals | **ReAct** | Flexibility to try different approaches |
| Complex dependencies | Hybrid (next tutorial!) | Best of both worlds |

---

### What's Next:

In the **Hybrid ReAct + Planner tutorial**, you'll learn how to combine:
- ReAct's adaptive reasoning
- Planner's parallel execution

Result: An agent that **thinks adaptively** but **executes efficiently**!

---

## 🔧 Try It Yourself!

Experiment with these challenges:

### Challenge 1: Simple Math Chain
```python
"Calculate 10 factorial, then count how many digits are in the result"
```

### Challenge 2: Web Research
```python
"Find the population of Tokyo and New York, then calculate which is larger by what percentage"
```

### Challenge 3: Multi-Agent Exploration
```python
"Search for the latest Python version, count the words in the search result, then write code to multiply that word count by 10"
```

### Challenge 4: Adaptive Discovery
```python
"Find information about Flyte workflow orchestration, then summarize it in exactly 50 words"
```

**Watch how the agent:**
- Breaks down each problem differently
- Chooses appropriate agents
- Adapts if first attempts don't work
- Reflects on each result

Run your own experiments below:

In [ ]:
# Your experiment here!
!python -m workflows.flyte_react --local --request "Your custom goal here" --max-steps 8

---

## 🙏 Summary

You've learned how to build **adaptive AI agents** using the ReAct pattern!

**Key concepts:**
- 🔄 Iterative reasoning loop (Think → Act → Observe → Reflect)
- 🧠 Context accumulation across steps
- 🎯 Goal-oriented execution with early termination
- 🛡️ Robust handling of LLM outputs
- 📊 Full traceability of decision-making

**The power of ReAct:**
> "The best plan is being able to change your plan based on what you learn."

Ready for the next level? Check out the **Hybrid ReAct + Planner tutorial** to combine adaptive reasoning with parallel execution!

---

Questions? Experiments? Share what you built! 🚀